In [1]:
from pathlib import Path
import pandas as pd
import emoji
import numpy as np
import re
import unicodedata
import html
import string

In [2]:
# Prepare folders
# if the folder structure not exists, then i will create
DATA_RAW = Path("data/raw"); DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED = Path("data/processed"); DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

In [3]:
CSV_PATH = DATA_RAW / "dataset.csv"
df = pd.read_csv(CSV_PATH, encoding="utf-8")

In [4]:
print("\n=== Columns ===")
print(df.columns.tolist())

print("\nShape:", df.shape)

print("\nNulls per column:")
print(df.isna().sum())

if "Sentiment" in df.columns:
    print("\nSentiment distribution:")
    print(df["Sentiment"].value_counts(dropna=False))

if "Emotion" in df.columns:
    print("\nEmotion distribution:")
    print(df["Emotion"].value_counts(dropna=False))

if "Product Name" in df.columns:
    print("\nUnique Product Names:", df["Product Name"].nunique())
if "Product Category" in df.columns:
    print("Unique Product Categories:", df["Product Category"].nunique())
if "Review" in df.columns:
    print("\nUnique review distribution:", df["Review"].nunique())


=== Columns ===
['Rating', 'Review', 'Product Name', 'Product Category', 'Emotion', 'Data Source', 'Sentiment']

Shape: (78130, 7)

Nulls per column:
Rating              0
Review              0
Product Name        0
Product Category    0
Emotion             0
Data Source         0
Sentiment           0
dtype: int64

Sentiment distribution:
Sentiment
Positive    67268
Negative    10862
Name: count, dtype: int64

Emotion distribution:
Emotion
Happy      46635
Love       20633
Sadness     6533
Anger       3171
Fear        1158
Name: count, dtype: int64

Unique Product Names: 2476
Unique Product Categories: 152

Unique review distribution: 64147


In [5]:
df_dedup = df.drop_duplicates(subset=["Review"], keep="first").reset_index(drop=True)

In [6]:
data_dedup_path = DATA_PROCESSED / "dataset_dedup.csv"
df_dedup.to_csv(data_dedup_path, index=False, encoding="utf-8")

In [7]:
print("After dedup:", len(df_dedup))
print("Removed:", len(df) - len(df_dedup))

print("\nShape:", df_dedup.shape)

if "Sentiment" in df_dedup.columns:
    print("\nSentiment distribution:")
    print(df_dedup["Sentiment"].value_counts(dropna=False))

if "Emotion" in df_dedup.columns:
    print("\nEmotion distribution:")
    print(df_dedup["Emotion"].value_counts(dropna=False))

if "Product Name" in df_dedup.columns:
    print("\nUnique Product Names:", df_dedup["Product Name"].nunique())
if "Product Category" in df_dedup.columns:
    print("Unique Product Categories:", df_dedup["Product Category"].nunique())
if "Review" in df_dedup.columns:
    print("\nUnique review distribution:", df_dedup["Review"].nunique())

After dedup: 64147
Removed: 13983

Shape: (64147, 7)

Sentiment distribution:
Sentiment
Positive    53692
Negative    10455
Name: count, dtype: int64

Emotion distribution:
Emotion
Happy      37654
Love       16038
Sadness     6285
Anger       3047
Fear        1123
Name: count, dtype: int64

Unique Product Names: 2361
Unique Product Categories: 143

Unique review distribution: 64147


In [8]:
TEXT_COL = "Review"  # change if needed

# df = pd.read_csv(IN_PATH, encoding="utf-8")

# If you want to treat "😍😍!!!" as emoji-only, keep punctuation removal ON.
ALLOW_PUNCT = True

PUNCT_SPACES_RE = re.compile(r"[\s\.\,\!\?\:\;\-\_\(\)\[\]\{\}\'\"\+\=\*\/\\\|@#\$%\^&~`]+")

def is_emoji_only(text: str) -> bool:
    if not isinstance(text, str):
        return False
    s = text.strip()
    if s == "":
        return True  # if you also want to remove empty reviews, change to True

    # remove all emojis
    s_no_emoji = emoji.replace_emoji(s, replace="").strip()

    # optionally remove punctuation/spaces too (so "😍😍!!!" becomes empty)
    if ALLOW_PUNCT:
        s_no_emoji = PUNCT_SPACES_RE.sub("", s_no_emoji).strip()

    # If nothing left => emoji-only
    return s_no_emoji == ""

mask = df_dedup[TEXT_COL].apply(is_emoji_only)

print("Total rows after dedup:", len(df_dedup))
print("Emoji-only rows:", int(mask.sum()))
print("\nExamples (will be removed):")
print(df_dedup.loc[mask, TEXT_COL].head(20).to_list())


Total rows after dedup: 64147
Emoji-only rows: 114

Examples (will be removed):
['👍', '💯💯💯', '🔥', '😍😍😍', '👍👍👍👍👍👍', '🥰🥰🥰🥰🥰🥰', '😊😊😊', '😎', '💞💞🥀', '🥰💚🍀', '😍🥰🥰', '💯💯', '😙😙😙😙', '😍😍', '🙂', '👍👍👍', '❤️❤️', '🥰🖤', '💖💖', '♥️♥️♥️♥']


In [11]:
df_dedup_emoji_path = DATA_PROCESSED / "dataset_dedup_emoji.csv"
df_dedup_emoji = df_dedup.loc[~mask].reset_index(drop=True)
df_dedup_emoji.to_csv(df_dedup_emoji_path, index=False, encoding="utf-8")

print("\nAfter removal:", len(df_dedup_emoji))


After removal: 64033


In [12]:
if "Sentiment" in df_dedup_emoji.columns:
    print("\nSentiment distribution:")
    print(df_dedup_emoji["Sentiment"].value_counts(dropna=False))

if "Emotion" in df_dedup_emoji.columns:
    print("\nEmotion distribution:")
    print(df_dedup_emoji["Emotion"].value_counts(dropna=False))

if "Product Name" in df_dedup_emoji.columns:
    print("\nUnique Product Names:", df_dedup_emoji["Product Name"].nunique())
if "Product Category" in df_dedup_emoji.columns:
    print("Unique Product Categories:", df_dedup_emoji["Product Category"].nunique())
if "Review" in df_dedup_emoji.columns:
    print("\nUnique review distribution:", df_dedup_emoji["Review"].nunique())


Sentiment distribution:
Sentiment
Positive    53586
Negative    10447
Name: count, dtype: int64

Emotion distribution:
Emotion
Happy      37606
Love       15980
Sadness     6282
Anger       3043
Fear        1122
Name: count, dtype: int64

Unique Product Names: 2361
Unique Product Categories: 143

Unique review distribution: 64033


In [16]:
# ---------------- Email ----------------
EMAIL_RE = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.IGNORECASE)

# ---------------- URLs ----------------
URL_SCHEME_RE = re.compile(r"\b(?:https?://|ftp://)\S+\b", re.IGNORECASE)
URL_WWW_RE    = re.compile(r"\bwww\.\S+\b", re.IGNORECASE)
BARE_DOMAIN_RE = re.compile(
    r"\b(?:[a-z0-9-]+\.)+(?:com|net|org|io|co|bd|uk|us|in|me|app|dev|ai|shop|store|xyz|info)\b(?:/\S*)?",
    re.IGNORECASE
)

# ---------------- Phone numbers ----------------
BD_PHONE_RE = re.compile(r"(?<!\d)(?:\+?88)?01[3-9]\d{8}(?!\d)")
PHONE_CANDIDATE_RE = re.compile(r"(?:\+?\d[\d\-\s\(\)]{6,}\d)")

def has_phone(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.strip()
    if t == "":
        return False

    if BD_PHONE_RE.search(t):
        return True

    for m in PHONE_CANDIDATE_RE.finditer(t):
        digits = re.sub(r"\D", "", m.group(0))
        if len(digits) >= 9:   # make 10 for stricter
            return True
    return False

# ---------------- Order / Tracking / Invoice IDs (English + Bangla) ----------------
ORDER_KEYWORDS_RE = re.compile(
    r"\b("
    # English
    r"order\s*(id|no|number)|"
    r"invoice\s*(id|no|number)|"
    r"tracking\s*(id|no|number)|"
    r"track\s*(id|no|number)|"
    r"consignment\s*(id|no|number)|"
    r"parcel\s*(id|no|number)|"
    r"shipment\s*(id|no|number)|"
    r"waybill|awb|"
    r"reference\s*(id|no|number)|"
    r"ref\s*(id|no|number)"
    r")\b"
    r"|"
    r"(অর্ডার|অর্ডার\s*আইডি|অর্ডার\s*নং|অর্ডার\s*নম্বর|অর্ডার\s*নাম্বার|"
    r"ইনভয়েস|ইনভয়েস|ইনভয়েস\s*আইডি|ইনভয়েস\s*আইডি|ইনভয়েস\s*নং|ইনভয়েস\s*নং|"
    r"ট্র্যাকিং|ট্র্যাকিং\s*আইডি|ট্র্যাকিং\s*নং|ট্র্যাকিং\s*নম্বর|"
    r"কনসাইনমেন্ট|কনসাইনমেন্ট\s*নং|কনসাইনমেন্ট\s*নম্বর|"
    r"পার্সেল|পার্সেল\s*নং|পার্সেল\s*নম্বর|"
    r"শিপমেন্ট|শিপমেন্ট\s*আইডি|"
    r"ওয়ে বিল|ওয়েবিল|ওয়ে বিল|"
    r"রেফারেন্স|রেফ\s*আইডি|রেফারেন্স\s*নং|রেফারেন্স\s*নম্বর)"
    ,
    re.IGNORECASE
)

# keyword then an ID-like token nearby (works for Bangla keywords too)
ORDER_ID_NEAR_KEYWORD_RE = re.compile(
    r"("  # keyword group (English or Bangla)
    r"(?:order\s*(?:id|no|number)|invoice\s*(?:id|no|number)|tracking\s*(?:id|no|number)|"
    r"consignment\s*(?:id|no|number)|shipment\s*(?:id|no|number)|waybill|awb|reference\s*(?:id|no|number)|ref\s*(?:id|no|number))"
    r"|"
    r"(?:অর্ডার\s*আইডি|অর্ডার\s*নং|অর্ডার\s*নম্বর|ইনভয়েস\s*আইডি|ইনভয়েস\s*আইডি|ইনভয়েস\s*নং|ইনভয়েস\s*নং|"
    r"ট্র্যাকিং\s*আইডি|ট্র্যাকিং\s*নং|ট্র্যাকিং\s*নম্বর|কনসাইনমেন্ট\s*নং|কনসাইনমেন্ট\s*নম্বর|"
    r"পার্সেল\s*নং|পার্সেল\s*নম্বর|শিপমেন্ট\s*আইডি|ওয়ে বিল|ওয়েবিল|ওয়ে বিল|রেফারেন্স\s*নং|রেফারেন্স\s*নম্বর|রেফ\s*আইডি)"
    r")"
    r"\s*[:#\-]?\s*"
    r"([A-Z0-9][A-Z0-9\-_/]{3,})",  # ID token
    re.IGNORECASE
)

DIGITS_HEAVY_TOKEN_RE = re.compile(r"\b[A-Z0-9\-_/]{5,}\b", re.IGNORECASE)

def looks_like_id(token: str) -> bool:
    digits = sum(ch.isdigit() for ch in token)
    return digits >= 6  # requires enough digits to reduce false positives

def has_order_id(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.strip()
    if t == "":
        return False

    m = ORDER_ID_NEAR_KEYWORD_RE.search(t)
    if m and looks_like_id(m.group(2)):
        return True

    if ORDER_KEYWORDS_RE.search(t):
        for tok in DIGITS_HEAVY_TOKEN_RE.findall(t):
            if looks_like_id(tok):
                return True

    return False

# ---------------- Combined filter ----------------
def should_remove(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.strip()
    if t == "":
        return False
    return (
        EMAIL_RE.search(t) is not None
        or URL_SCHEME_RE.search(t) is not None
        or URL_WWW_RE.search(t) is not None
        or BARE_DOMAIN_RE.search(t) is not None
        or has_phone(t)
        or has_order_id(t)
    )

mask = df_dedup_emoji[TEXT_COL].apply(should_remove)

print("Total rows after dedup & emoji:", len(df_dedup_emoji))
print("Rows removed (email/url/phone/order-id):", int(mask.sum()))

print("\nExamples (will be removed):")
print(df_dedup_emoji.loc[mask, TEXT_COL].head(30).to_list())


Total rows: 64033
Rows removed (email/url/phone/order-id): 87

Examples (will be removed):
['কেনার আগে দেখে আসুন ভিডিও।। সাচ করুন PRB786 লিখে youtube,facebook,google,tiktok, url link -https://youtube.com/shorts/HRgKsgam3p0?si=Kn9n93Gr8h076ggE', 'অর্ডিনারি আইটিতে ordinaryit.com এ ফ্রিল্যান্সিং লেখালেখির কাজ করার জন্য নিয়েছি অনেক সুন্দর কাজ করছে।', 'প্লে বাটনে কাজ করতেছে না আমি আজ রিটার্ন করব।01819183039', 'খুব ভালো ক্যামেরা সবাই কিনতে পারেন 🌹🌹 ধন্যবাদ ডিলিভেরি ভাইকে তার কথা অনেক ভালো লাগলো🌹🌹আমি wifi connect করতে পারতেছি না কেও পারলে একটু বলবেন 01902022833 আমার ইমু আছে', 'OTG ta je kajer Jonno niye silam se kajta Korte pari nai.sudhu mobile change dite parci.amar kache to changer already asei.ai OTG ta sudhu changer lagi nite paren, Thanks for seller,😉', 'U deserve 5 star … thank u soo much … \n\nU all can be buy from me on daraz … visit my store .. \n\nhttps://www.daraz.com.bd/shop/barakat-al-madina-shop?liteShop=true&spm=a2a0e.store_hp.top.share&dsource=share&laz_share_info=14651875_1

In [ ]:
df_dedup_emoji_contact = df_dedup_emoji.loc[~mask].reset_index(drop=True)
df_dedup_emoji_contact.to_csv(DATA_PROCESSED/ "dataset_dedup_emoji_contact.csv", index=False, encoding="utf-8")

print("\nAfter removal of dedup, emoji, contact info, order id:", len(df_dedup_emoji_contact))


In [24]:
# ---------------- ABSA suitability filter (remove generic / too-short / non-aspect reviews) ----------------
# Goal: keep more ABSA-friendly reviews (multi-aspect or compound/contrast), remove generic ones like
# "Recommend", "Good", "Awesome", "ভালো", "very good product", etc.

ABSA_FILTER_LEVEL = "strict"  # "strict" (recommended) or "balanced"
OUT_ABSA_PATH = DATA_PROCESSED / f"dataset_absa_ready_{ABSA_FILTER_LEVEL}.csv"

# --- helpers ---
SPACE_RE = re.compile(r"\s+")
PUNCT_KEEP_CLAUSE_SEP_RE = re.compile(r"[^\w\s\u0980-\u09FF,;।!?]")  # keep bn letters + common clause separators

def _strip_emoji(s: str) -> str:
    # uses emoji module if available (already imported above)
    try:
        return emoji.replace_emoji(s, replace=" ")
    except Exception:
        return s

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = html.unescape(s)
    s = unicodedata.normalize("NFKC", s)
    s = _strip_emoji(s)
    s = s.lower()
    s = PUNCT_KEEP_CLAUSE_SEP_RE.sub(" ", s)
    s = SPACE_RE.sub(" ", s).strip()
    return s

# Generic phrases/words (you can extend this list)
GENERIC_PHRASES = {
    "recommend", "recommended", "good", "very good", "nice", "awesome", "perfect", "fine",
    "thanks", "thank you", "just wow", "wow", "gd", "too good", "not bad",
    "good product", "very good product", "good quality product", "good stuff", "looks great",
    "valo", "bhalo", "khub valo", "khub bhalo", "valo na", "bhalo na",
    "ভালো", "ভাল", "খুব ভালো", "খুব ভাল", "ধন্যবাদ", "অসাধারণ", "দারুন", "চমৎকার", "জোস",
    "মোটামুটি", "মন্দ না", "ভালো না", "খারাপ"
}

GENERIC_TOKENS = {
    # English
    "good", "great", "nice", "awesome", "perfect", "fine", "excellent", "amazing", "wow",
    "recommend", "recommended", "thanks", "thank", "satisfied", "love", "happy", "best", "ok", "okay", "gd",
    "product", "quality", "stuff", "looks",
    # Romanized Bangla
    "valo", "bhalo", "khub", "onek", "sundor", "joss", "darun", "motamoti", "baje",
    # Bangla
    "ভালো", "ভাল", "খুব", "অসাধারণ", "দারুন", "চমৎকার", "ধন্যবাদ", "জোস", "সুন্দর", "মোটামুটি", "খারাপ", "বাজে", "মন্দ"
}

STOPWORDS = set("""
 a an the is are was were am be been being to of in on for at by with and or but if then so as from this that it its
 i we you he she they my our your their me us him her them
 আমি আমরা তুমি তোমরা সে তারা এই ওই সেটা এটা ওটা এবং আর কিন্তু তো তবে যদি তাহলে থেকে জন্য সাথে উপর মধ্যে দিয়ে যে না খুব অনেক একদম শুধু
 ami amra tumi tora apni apnara she tara eta eita oita ar ebong kintu tobe jodio na khub onek sudhu
""".split())

# Aspect lexicon: categories -> keywords (English + Bangla + common Banglish)
# Aspect lexicon: taxonomy-aligned categories -> keywords (English + Bangla + common Banglish)
# NOTE: this is a lightweight keyword lexicon used ONLY for filtering ABSA-suitable reviews.
# It is NOT the final taxonomy mapping for ABSA labeling.
ASPECTS = {
    # -------- Global aspects --------
    "DELIVERY": [
        "delivery", "delivered", "deliver", "shipping", "ship", "courier", "rider", "dispatch", "arrival", "arrive", "received", "receive",
        "on time", "same day", "next day", "late", "delay", "delayed", "slow", "fast",
        "ডেলিভারি", "ডেলিভারি ম্যান", "কুরিয়ার", "কুরিয়ার", "কুরিয়ার সার্ভিস", "রাইডার", "ডিসপ্যাচ", "পাঠানো", "পাঠিয়েছে", "পৌঁছ", "পৌঁছেছে",
        "সময়", "সময়", "সময়মতো", "সময়মতো", "দেরি", "লেট", "দ্রুত", "ধীরে"
    ],
    "PACKAGING": [
        "packaging", "package", "packed", "pack", "box", "carton", "wrapper", "seal", "sealed", "bubble wrap", "tape",
        "protection", "protect", "safe", "damaged", "broken", "dent", "crushed", "leak", "leakage", "torn",
        "pouch", "poly", "polythene",
        "প্যাকেজিং", "প্যাকেট", "বক্স", "কার্টন", "র‌্যাপ", "র‍্যাপ", "মোড়ক", "মোড়ক", "সিল", "সিলগালা", "টেপ",
        "নিরাপদ", "প্রটেকশন", "ক্ষতিগ্রস্ত", "নষ্ট", "ভাঙা", "ডেন্ট", "চিপ", "চাপা", "ফাটা", "লিক", "ছিদ্র"
    ],
    "PRICE": [
        "price", "pricing", "cost", "value", "worth", "value for money", "affordable", "cheap", "cheaper", "expensive", "overpriced",
        "budget", "discount", "offer", "deal", "sale", "taka", "tk", "৳",
        "দাম", "মূল্য", "টাকা", "সস্তা", "কমদাম", "বাজেট", "ডিসকাউন্ট", "ছাড়", "অফার", "বেশি দাম", "দাম বেশি"
    ],
    "SERVICE": [
        "service", "seller", "shop", "store", "support", "help", "customer care", "after sales", "response", "reply", "communication",
        "behavior", "behaviour", "polite", "rude", "helpful",
        "return", "refund", "replacement", "exchange", "complain", "complaint",
        "warranty", "guarantee", "guaranty",
        "সার্ভিস", "সেলার", "বিক্রেতা", "শপ", "স্টোর", "সাপোর্ট", "হেল্প", "কাস্টমার কেয়ার", "কাস্টমার", "রেসপন্স", "রিপ্লাই",
        "ব্যবহার", "আচরণ", "ভদ্র", "রূঢ়", "খারাপ ব্যবহার",
        "রিটার্ন", "রিফান্ড", "রিপ্লেস", "চেঞ্জ", "এক্সচেঞ্জ", "অভিযোগ",
        "ওয়ারেন্টি", "ওয়ারেন্টি", "গ্যারান্টি"
    ],

    # -------- Electronics & Gadgets --------
    "PERFORMANCE": [
        "performance", "speed", "smooth", "lag", "slow", "hang", "freeze", "responsive",
        "heating", "overheat", "temperature", "gaming", "multitask", "processor", "chipset",
        "পারফরম্যান্স", "স্পিড", "স্মুথ", "ল্যাগ", "স্লো", "হ্যাং", "ফ্রিজ", "রেসপন্সিভ",
        "গরম", "হিটিং", "ওভারহিট", "টেম্পারেচার", "গেমিং", "প্রসেসর"
    ],
    "BATTERY": [
        "battery", "backup", "charge", "charging", "fast charging", "slow charging", "charger", "power", "drain", "standby",
        "ব্যাটারি", "ব্যাকআপ", "চার্জ", "চার্জিং", "ফাস্ট চার্জ", "চার্জার", "পাওয়ার", "ড্রেন", "চার্জ ধরে"
    ],
    "DISPLAY": [
        "display", "screen", "touchscreen", "touch", "lcd", "led", "amoled", "oled",
        "resolution", "brightness", "dim", "color", "colour", "refresh", "hz", "viewing angle",
        "pixel", "dead pixel", "scratch",
        "ডিসপ্লে", "স্ক্রিন", "টাচ", "টাচস্ক্রিন", "এলসিডি", "এলইডি", "অ্যামোলেড", "ওলেড",
        "রেজোলিউশন", "ব্রাইটনেস", "ডিম", "কালার", "রিফ্রেশ", "ভিউ অ্যাঙ্গেল",
        "পিক্সেল", "ডেড পিক্সেল", "স্ক্র্যাচ"
    ],
    "CAMERA": [
        "camera", "photo", "picture", "image", "video", "selfie", "portrait", "night mode", "zoom", "focus", "stabilization",
        "photo quality", "video quality",
        "ক্যামেরা", "ছবি", "ফটো", "পিকচার", "ইমেজ", "ভিডিও", "সেলফি", "পোর্ট্রেট", "নাইট মোড", "জুম", "ফোকাস",
        "ফটো কোয়ালিটি", "ভিডিও কোয়ালিটি"
    ],
    "BUILD_QUALITY": [
        "build quality", "build", "material", "sturdy", "strong", "durable", "finishing", "finish", "assembly",
        "plastic", "metal", "glass", "hinge",
        "বিল্ড কোয়ালিটি", "ম্যাটেরিয়াল", "মজবুত", "শক্ত", "টেকসই", "ফিনিশিং", "অ্যাসেম্বলি",
        "প্লাস্টিক", "মেটাল", "ধাতব", "গ্লাস", "হিঞ্জ"
    ],
    "AUDIO": [
        "audio", "sound", "speaker", "loud", "volume", "mic", "microphone", "call quality", "bass", "noise", "distortion",
        "earphone", "headphone",
        "অডিও", "সাউন্ড", "স্পিকার", "ভলিউম", "মাইক", "কল কোয়ালিটি", "বেস", "নয়েজ", "ডিস্টরশন",
        "ইয়ারফোন", "হেডফোন"
    ],
    "SOFTWARE": [
        "software", "os", "android", "ios", "update", "bug", "issue", "ui", "interface", "settings", "reset",
        "language", "keyboard", "app crash", "crash",
        "সফটওয়্যার", "আপডেট", "বাগ", "ইস্যু", "ইউআই", "ইন্টারফেস", "সেটিংস", "রিসেট",
        "ভাষা", "কিবোর্ড", "ক্র্যাশ"
    ],
    "STORAGE": [
        "storage", "memory", "rom", "space", "internal storage", "sd card", "expandable",
        "স্টোরেজ", "মেমোরি", "স্পেস", "ইন্টারনাল", "এসডি কার্ড", "এক্সপেন্ডেবল"
    ],
    "AUTHENTICITY": [
        "original", "genuine", "authentic", "real", "sealed", "brand new",
        "fake", "replica", "copy", "duplicate",
        "অরিজিনাল", "জেনুইন", "আসল", "অথেনটিক", "সিল", "নতুন",
        "নকল", "ফেক", "রেপ্লিকা", "কপি"
    ],

    # -------- Fashion & Apparel --------
    "MATERIAL": [
        "material", "fabric", "cloth", "cotton", "polyester", "silk", "chiffon", "denim", "leather", "faux leather",
        "stretch", "breathable",
        "ম্যাটেরিয়াল", "কাপড়", "কাপড়", "সুতি", "কটন", "পলিয়েস্টার", "সিল্ক", "শিফন", "ডেনিম", "চামড়া", "লেদার",
        "স্ট্রেচ", "ব্রিদেবল"
    ],
    "FIT_SIZE": [
        "size", "fit", "fitting", "loose", "tight", "small", "large", "length", "measurement", "waist", "shoulder", "sleeve",
        "xl", "xxl",
        "সাইজ", "মাপ", "ফিট", "ফিটিং", "ঢিলা", "টাইট", "ছোট", "বড়", "লম্বা", "মেজারমেন্ট", "হাতা", "কাঁধ"
    ],
    "DESIGN": [
        "design", "style", "look", "pattern", "print", "color", "colour", "finishing", "stitch", "stitching",
        "beautiful", "pretty",
        "ডিজাইন", "স্টাইল", "লুক", "প্যাটার্ন", "প্রিন্ট", "রং", "কালার", "ফিনিশিং", "সেলাই",
        "সুন্দর"
    ],
    "DURABILITY": [
        "durable", "long lasting", "tear", "torn", "fade", "faded", "shrink", "shrunk", "wash", "washing",
        "টেকসই", "টেকে", "ছিঁড়ে", "ফাটা", "রং উঠে", "ফিকে", "সঙ্কুচিত", "ধোয়া", "ধুয়ে"
    ],
    "FUNCTIONALITY": [
        "comfortable", "comfort", "pocket", "zip", "zipper", "button", "buttons", "waterproof", "warm", "soft",
        "আরাম", "আরামদায়ক", "পকেট", "জিপ", "বোতাম", "ওয়াটারপ্রুফ", "গরম", "নরম"
    ],

    # -------- Beauty & Personal Care --------
    "EFFICACY": [
        "effective", "works", "working", "result", "improve", "improved", "reduce", "treatment",
        "acne", "pimple", "spot", "glow", "glowing", "brighten", "whitening",
        "hair fall", "dandruff",
        "কার্যকর", "কাজ করে", "ফল", "রেজাল্ট", "উন্নতি", "কমায়",
        "ব্রণ", "পিম্পল", "দাগ", "গ্লো", "উজ্জ্বল", "ফর্সা",
        "চুল পড়া", "খুশকি"
    ],
    "TEXTURE_SMELL": [
        "smell", "scent", "fragrance", "perfume", "odor", "texture",
        "sticky", "oily", "greasy", "dry", "absorb", "lightweight",
        "গন্ধ", "সুগন্ধ", "দুর্গন্ধ", "দুর্গন্ধ", "টেক্সচার",
        "চিটচিটে", "তেলতেলে", "গ্রিসি", "শুষ্ক", "শোষে", "হালকা"
    ],
    "SUITABILITY": [
        "suitable", "suit", "skin type", "sensitive", "allergy", "irritation", "rash",
        "dry skin", "oily skin",
        "উপযোগী", "স্কিন টাইপ", "সেন্সিটিভ", "অ্যালার্জি", "জ্বালা", "র‍্যাশ",
        "ড্রাই স্কিন", "অয়েলি স্কিন"
    ],

    # -------- Home, Living & Appliances --------
    "AESTHETICS": [
        "aesthetic", "aesthetics", "look", "looks", "beautiful", "premium", "modern", "design",
        "ডেকর", "ডেকোর", "দেখতে", "লুক", "সুন্দর", "প্রিমিয়াম", "আধুনিক"
    ],
    "INSTALLATION": [
        "install", "installation", "setup", "set up", "mount", "fit", "fitting", "manual", "instruction",
        "screw", "bracket",
        "ইনস্টল", "ইনস্টলেশন", "সেটআপ", "লাগানো", "ফিটিং", "ম্যানুয়াল", "নির্দেশনা",
        "স্ক্রু", "ব্র্যাকেট"
    ],
    "DIMENSIONS": [
        "dimension", "dimensions", "size", "width", "height", "depth", "capacity", "liter", "litre", "inch", "cm", "mm",
        "মাপ", "ডাইমেনশন", "দৈর্ঘ্য", "প্রস্থ", "উচ্চতা", "ক্যাপাসিটি", "লিটার", "ইঞ্চি", "সেমি", "মিমি"
    ],

    # -------- Food & Grocery --------
    "TASTE_FLAVOR": [
        "taste", "flavor", "flavour", "delicious", "tasty", "spicy", "hot", "sweet", "salty", "bitter",
        "স্বাদ", "টেস্ট", "মজা", "ঝাল", "গরম", "মিষ্টি", "নোনতা", "তিতা"
    ],
    "FRESHNESS": [
        "fresh", "freshness", "stale", "rotten", "spoiled", "smelly", "crispy",
        "টাটকা", "ফ্রেশ", "বাসি", "পচা", "নষ্ট", "দুর্গন্ধ", "কুড়কুড়ে", "কুড়কুড়ে নয়"
    ],
    "QUALITY": [
        "quality", "purity", "pure", "ingredients", "authentic", "adulterated",
        "মান", "খাঁটি", "পিওর", "উপাদান", "ভেজাল", "অথেনটিক"
    ],
    "EXPIRY": [
        "expiry", "expire", "expired", "best before", "mfg", "manufacture date", "date",
        "মেয়াদ", "মেয়াদ", "এক্সপায়ারি", "এক্সপায়ারি", "তারিখ", "উৎপাদন"
    ],

    # -------- Automotive --------
    "COMPATIBILITY": [
        "compatible", "compatibility", "fitment", "fit", "model match", "model", "match", "fits",
        "car", "bike", "motorcycle",
        "কম্প্যাটিবল", "কম্প্যাটিবিলিটি", "ফিটমেন্ট", "মিল", "মিলে", "মডেল", "ফিট", "ফিট হয়",
        "গাড়ি", "বাইক", "মোটর"
    ],
}

COMPOUND_MARKERS = set(
    "but however though although yet still except whereas kintu tobe jodio ar ebong"
    " কিন্তু তবে যদিও অথচ তবুও এবং আর".split()
)

def aspect_categories(norm: str) -> set:
    hits = set()
    for cat, kws in ASPECTS.items():
        for kw in kws:
            if kw in norm:
                hits.add(cat)
                break
    return hits

def content_token_count(norm: str) -> int:
    toks = norm.split()
    c = 0
    for t in toks:
        if t in STOPWORDS:
            continue
        if t in GENERIC_TOKENS:
            continue
        if t.isdigit():
            continue
        if len(t) <= 2:
            continue
        c += 1
    return c

def is_generic(norm: str) -> bool:
    if not norm:
        return True
    if norm in GENERIC_PHRASES:
        return True
    toks = norm.split()
    cc = content_token_count(norm)
    asp = aspect_categories(norm)

    # Remove very short generic lines like: "good", "awesome", "ভালো", "very good product"
    if len(toks) <= 3 and cc < 2 and len(asp) == 0:
        return True

    # If mostly generic tokens and no aspects, remove
    if len(toks) <= 6 and len(asp) == 0:
        ratio_generic = sum(t in GENERIC_TOKENS for t in toks) / max(1, len(toks))
        if ratio_generic >= 0.6 and cc < 2:
            return True

    return False

def is_compound(norm: str, raw: str) -> bool:
    # marker-based
    toks = set(norm.split())
    if toks & COMPOUND_MARKERS:
        return True
    # punctuation-based (often indicates multiple clauses)
    if any(sep in raw for sep in [",", ";", "।", "!", "?"]):
        return True
    return False

def keep_for_absa(raw: str) -> bool:
    norm = normalize_text(raw)
    if is_generic(norm):
        return False

    toks = norm.split()
    cats = aspect_categories(norm)
    cc = content_token_count(norm)

    # Needs at least some aspect cue OR at least 2 non-generic content tokens
    has_aspect = (len(cats) > 0) or (cc >= 2)
    if not has_aspect:
        return False

    if ABSA_FILTER_LEVEL == "balanced":
        # Keep: compound OR detailed OR aspect-specific
        if is_compound(norm, raw) and len(toks) >= 5:
            return True
        if len(toks) >= 8:
            return True
        if len(cats) > 0 and len(toks) >= 3:
            return True
        if cc >= 3 and len(toks) >= 5:
            return True
        return False

    # STRICT (recommended): keep multi-aspect OR (compound + aspect + enough length)
    if len(cats) >= 2:
        return True
    if len(cats) >= 1 and is_compound(norm, raw) and len(toks) >= 8:
        return True
    return False

mask_absa = df_dedup_emoji_contact[TEXT_COL].astype(str).apply(keep_for_absa)

df_absa_ready = df_dedup_emoji_contact.loc[mask_absa].reset_index(drop=True)
df_removed_absa = df_dedup_emoji_contact.loc[~mask_absa]

df_absa_ready.to_csv(OUT_ABSA_PATH, index=False, encoding="utf-8")

print("\n=== ABSA Filtering ===")
print("Filter level:", ABSA_FILTER_LEVEL)
print("Before:", len(df_dedup_emoji_contact))
print("Removed:", int((~mask_absa).sum()))
print("Kept:", len(df_absa_ready))
print("Saved:", OUT_ABSA_PATH)

print("\nExamples removed:")
print(df_removed_absa[TEXT_COL].head(25).to_list())

print("\nExamples kept:")
print(df_absa_ready[TEXT_COL].head(25).to_list())


=== ABSA Filtering ===
Filter level: strict
Before: 63946
Removed: 32502
Kept: 31444
Saved: data/processed/dataset_absa_ready_strict.csv

Examples removed:
['অল্প দামে দারুন একটা স্মার্টফোন 💙', 'Delay Delivery... Good Product.', 'Poor seller response', 'এখনো ভালো আছে ভবিষ্যতে কি হবে সেটা দেখার বিষয়', 'bought it for my mother..\n\nso far so good.', 'good product for me thanks seller', 'Fast delivery and good product.', 'good quality product', 'অসাধারণ,,,!!', 'good rider', 'Got it within one day, working good.', 'intact,original product', 'Darun phone', 'ink discount a pailam', 'varryGood  product...❤️\n100/100%  Original product. ❤️\nThankss DarazMoll .. Thankss Realme.❤️💝', 'realme c55 hate paye ank happy thanks daraz & realme k over all good', 'realme c55 hate paye ank happy thanks daraz & realme k', 'অনেক পছন্দ হয়েছে', 'Thank you daraz for your fast delivary 🫰', 'valo..nite paren sobai...🤗', '১ দিনেই ডেলিভারি পেয়েছি', 'very good suggested', 'good you can take', 'thanks to daraz', "

In [29]:
WORD_RE = re.compile(r"[A-Za-z0-9\u0980-\u09FF]+")

def word_count(text):
    if not isinstance(text, str):
        return 0
    return len(WORD_RE.findall(text))

df_absa_ready["word_count"] = df_absa_ready[TEXT_COL].apply(word_count)
df_filtered = df_absa_ready[df_absa_ready["word_count"].between(4, 25, inclusive="both")].reset_index(drop=True)

print("Rows:", len(df_absa_ready))
print("Word count stats:")
print(df_absa_ready["word_count"].describe())
print("After :", len(df_filtered))
# print("\nTop 10 longest reviews by word count:")
# print(df_absa_ready.sort_values("word_count", ascending=False)[[TEXT_COL, "word_count"]].head(10).to_string(index=False))


Rows: 31444
Word count stats:
count    31444.000000
mean        20.056227
std         17.619293
min          1.000000
25%         10.000000
50%         15.000000
75%         25.000000
max        354.000000
Name: word_count, dtype: float64
After : 23458


In [32]:
df_filtered.to_csv(DATA_PROCESSED / f"dataset_absa_ready_{ABSA_FILTER_LEVEL}_wc_filtered.csv", index=False, encoding="utf-8")

In [31]:
if "Sentiment" in df_filtered.columns:
    print("\nSentiment distribution:")
    print(df_filtered["Sentiment"].value_counts(dropna=False))

if "Emotion" in df_filtered.columns:
    print("\nEmotion distribution:")
    print(df_filtered["Emotion"].value_counts(dropna=False))

if "Product Name" in df_filtered.columns:
    print("\nUnique Product Names:", df_filtered["Product Name"].nunique())
if "Product Category" in df_filtered.columns:
    print("Unique Product Categories:", df_filtered["Product Category"].nunique())
if "Review" in df_filtered.columns:
    print("\nUnique review distribution:", df_filtered["Review"].nunique())


Sentiment distribution:
Sentiment
Positive    19327
Negative     4131
Name: count, dtype: int64

Emotion distribution:
Emotion
Happy      14905
Love        4422
Sadness     2585
Anger       1108
Fear         438
Name: count, dtype: int64

Unique Product Names: 1993
Unique Product Categories: 141

Unique review distribution: 23458


In [35]:
import time
from rapidfuzz import fuzz

# Load your latest filtered dataset
# df_filtered = pd.read_csv("data/processed/dataset_absa_ready_strict_wc_filtered.csv")

def fuzzy_deduplicate(df, text_col="Review", threshold=100):
    """
    Removes fuzzy duplicates using a length-sorted window approach.
    Complexity is reduced from O(N^2) to roughly O(N * window_size).
    """
    start_time = time.time()
    print(f"Starting fuzzy deduplication on {len(df)} rows...")

    # 1. Create a working copy and add a length column for sorting
    df_proc = df.copy()
    df_proc["_len"] = df_proc[text_col].astype(str).str.len()

    # 2. Sort by length: matching text usually has similar length
    # This groups "Good product" and "Good product." together
    df_proc = df_proc.sort_values("_len").reset_index(drop=True)

    to_drop = set()
    texts = df_proc[text_col].astype(str).tolist()
    n = len(texts)

    # 3. Iterate through rows
    for i in range(n):
        if i in to_drop:
            continue

        current_text = texts[i]
        current_len = len(current_text)

        # Optimization: Stop looking if the next row is too much longer to match
        # If threshold is 90%, length difference > 15-20% usually implies a mismatch.
        # We use a loose buffer here.
        max_len_diff = int(current_len * 0.2) + 5

        # Look ahead at the next 50 neighbors (Window Size)
        # You can increase range(i+1, min(i+50, n)) if you want to be more thorough
        for j in range(i + 1, min(i + 100, n)):
            if j in to_drop:
                continue

            next_text = texts[j]
            next_len = len(next_text)

            # Break window early if length difference is too large
            if next_len - current_len > max_len_diff:
                break

            # Calculate Similarity Ratio
            # fuzz.ratio is fast and handles punctuation/case diffs well.
            # Example: "Good phone" vs "Good phone." = ~95% match
            score = fuzz.ratio(current_text, next_text)

            if score >= threshold:
                to_drop.add(j)

        # Progress tracker
        if i % 5000 == 0 and i > 0:
            print(f"Processed {i}/{n} rows...")

    # 4. Remove duplicates and clean up
    df_final = df_proc.drop(index=list(to_drop)).drop(columns=["_len"])

    print(f"Finished in {time.time() - start_time:.2f}s")
    print(f"Removed {len(to_drop)} duplicates.")
    return df_final.reset_index(drop=True)

# --- EXECUTION ---
df_fuzzy_dedup = fuzzy_deduplicate(df_filtered, text_col="Review", threshold=90)

# Save result
df_fuzzy_dedup.to_csv("data/processed/dataset_fuzzy_dedup.csv", index=False)
print(f"New Shape: {df_fuzzy_dedup.shape}")

Starting fuzzy deduplication on 23458 rows...
Processed 5000/23458 rows...
Processed 10000/23458 rows...
Processed 15000/23458 rows...
Processed 20000/23458 rows...
Finished in 3.39s
Removed 50 duplicates.
New Shape: (23408, 8)


In [36]:
# --- STRATEGIC DOWNSAMPLING ---

# 1. Separate classes
df_pos = df_fuzzy_dedup[df_fuzzy_dedup["Sentiment"] == "Positive"]
df_neg = df_fuzzy_dedup[df_fuzzy_dedup["Sentiment"] == "Negative"]

# 2. Decide how many positives to keep (e.g., 2x the negatives)
target_neg_count = 4000
target_pos_count = target_neg_count * 2

print(f"Negatives (Original): {len(df_neg)}")
print(f"Negatives (Target): {target_neg_count}")
print(f"Positives (Original): {len(df_pos)}")
print(f"Positives (Target): {target_pos_count}")

# 3. Randomly sample the positives & negatives
df_neg_downsampled = df_neg.sample(n=target_neg_count, random_state=42)
df_pos_downsampled = df_pos.sample(n=target_pos_count, random_state=42)

# 4. Combine and Shuffle
df_balanced = pd.concat([df_pos_downsampled, df_neg_downsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Save final ready-to-train dataset
df_balanced.to_csv("data/processed/dataset_final_selected.csv", index=False)

print(f"\nFinal Dataset Shape: {df_balanced.shape}")
print(df_balanced["Sentiment"].value_counts())

Negatives (Original): 4129
Negatives (Target): 4000
Positives (Original): 19279
Positives (Target): 8000

Final Dataset Shape: (12000, 8)
Sentiment
Positive    8000
Negative    4000
Name: count, dtype: int64
